<a href="https://colab.research.google.com/github/argonism/TsukurinagaraRL/blob/master/Zerokara_chap6_Prioritized_Experience_Replay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prioritized Experience Replay

Q学習が進んでいないsのtransitionを優先的に学習する

## 優先基準
ベルマン方程式の絶対値誤差で優先順位をつける
$$Q(s_t, a_t) = R_{t+1} + \gamma Q(s_{t+1}, a_{t+1})$$

TD誤差と呼ぶ。（厳密にはTD誤差ではないらしい）
$$|[R(t+1) + \gamma \times max_a[Q(s(t+1), a)] - Q(s(t), a(t))$$

この誤差が大きい（学習が進んでいない）transitionを優先的にExperience Replay時に学習される。

メモリクラスと別に、TD誤差を保持するクラスを用意する。

学習の際は、TD誤差を確率としてtransitionからミニバッチ数取り出す。
0~TD誤差絶対値の総和の範囲で一様分布に従いミニバッチ分だけindexを取り出す。
- TD誤差の絶対値が大きさが選ばれやすさになる。
(同じtransitionが選ばれることもある?)

ただし、TD誤差が小さくなりすぎてtransitionが一切replayされないのを防ぐために、微小値TD_ERROR_EPSILONを足している。






In [ ]:
# 使用するパッケージのインストール
# gym==0.17.2 pyvirtualdisplay==1.3.2
# xvfb=2:1.19.6-1ubuntu4.4 python-opengl=3.1.0+dfsg-1 ffmpeg=7:3.4.8-0ubuntu0.2
# JSAnimation==0.1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install JSAnimation > /dev/null 2>&1

In [ ]:
 import numpy as np
 import matplotlib.pyplot as plt
 %matplotlib inline
 import gym
from gym.wrappers import Monitor

In [ ]:
# 動画の描画関数の宣言
import glob
import io
import os
import base64
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython import display as ipythondisplay
from IPython.display import HTML
from pyvirtualdisplay import Display

display = Display(visible=0, size=(640, 400))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def reset_video():
  mp4list = glob.glob('video/*.mp4')
  for mp4 in mp4list:
    os.remove(mp4)

def wrap_env(env):
  env = Monitor(env, './video', force=True, video_callable=(lambda ep: ep % 10 == 0))
  reset_video()
  return env

In [ ]:
class ReplayMemory:
  def __init__(self, CAPACITY):
    self.capacity = CAPACITY
    self.memory = []
    self.index = 0
  
  def push(self, state, action, state_next, reward):
    if len(self.memory) < self.capacity:
      self.memory.append(None)
    
    self.memory[self.index] = Transition(state, action, state_next, reward)

    self.index = (self.index + 1) % self.capacity
  
  def sample(self, batch_size):
    return random.sample(self.memory, batch_size)
  
  def __len__(self):
    return len(self.memory)


In [ ]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self, n_in, n_mid, n_out):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(n_in, n_mid)
    self.fc2 = nn.Linear(n_mid, n_mid)
    self.fc3 = nn.Linear(n_mid, n_out)
  
  def forward(self, x):
    h1 = F.relu(self.fc1(x))
    h2 = F.relu(self.fc2(h1))
    output = self.fc3(h2)
    return output

In [ ]:
TD_ERROR_EPSILON = 0.0001

class TDerrorMemory:
  def __init__(self, CAPACITY):
    self.capacity = CAPACITY
    self.memory = []
    self.index = 0

  def push(self, td_error):

    # この処理いる？
    if len(self.memory) < self.capacity:
      self.memory.append(None)

    self.memory[self.index] = td_error
    self.index = (self.index + 1) % self.capacity

  def __len__(self):
    return len(self.memory)
  
  def get_prioritized_indexes(self, batch_size):
    # TD誤差の総和を計算して、
    sum_absolute_td_error = np.sum(np.absolute(self.memory))
    sum_absolute_td_error += TD_ERROR_EPSILON * len(self.memory)

    # 0 ~ (TD誤差絶対値の総和)の範囲で、乱数をbatch_size個生成する。
    rand_list = np.random.uniform(0, sum_absolute_td_error, batch_size)
    rand_list = np.sort(rand_list)

    # 上の乱数を使って使うtransitionのindexを決めるわけだが、
    # 上の乱数は0 ~ TD誤差総和の範囲。
    # TD誤差をindex[0]から足して行って、その足して行った誤差が乱数を超えた時点のTD誤差に対応するindexを採用する。
    # これによってTD誤差が大きいほど採用される確率が高くなる。（そのTD誤差が採用されやすい -> TD誤差がより広い範囲をもつ -> TD誤差がより大きい)
    indexes = []
    idx = 0
    tmp_sum_absolute_td_error = 0
    for rand_num in rand_list:
      while tmp_sum_absolute_td_error < rand_num:
        tmp_sum_absolute_td_error += (
            abs(self.memory[idx]) + TD_ERROR_EPSILON
        )
        idx += 1
      if idx >= len(self.memory):
        idx = len(self.memory) - 1
      indexes.append(idx)

    return indexes

  def update_td_error(self, update_td_errors):
    self.memory = update_td_errors
  


In [ ]:
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

class Brain:
  def __init__(self, num_states, num_actions):
    self.num_actions = num_actions

    self.memory = ReplayMemory(CAPACITY)
    
    n_in, n_mid, n_out = num_states, 32, num_actions
    self.main_q_network = Net(n_in, n_mid, n_out)
    self.target_q_network = Net(n_in, n_mid, n_out)
    print(self.main_q_network)

    self.optimizer = optim.Adam(self.main_q_network.parameters(), lr=0.0001)

    self.td_error_memory = TDerrorMemory(CAPACITY)

  def replay(self, episode):
    '''保存した行動や結果、状態から結合パラメータを学習する'''

    if len(self.memory) < BATCH_SIZE:
      return
    
    # ミニバッチの作成
    self.batch, self.state_batch, self.action_batch, self.reward_batch, self.non_final_next_states = self.make_minibatch(episode)

    # 教師信号となるQ(s_t, a_t)を求める
    self.expected_state_action_values = self.get_expected_state_action_values()

    # 訓練
    self.update_main_q_network()

  def decide_action(self, state, episode):
    epsilon = 0.5 * (1 / (episode + 1))

    if epsilon <= np.random.uniform(0, 1):
      self.main_q_network.eval()
      with torch.no_grad():
        action = self.main_q_network(state).max(1)[1].view(1, 1)
        # view(1, 1)は、[torch.LongTensor of size 1]をsize 1x1にするらしい。逆にsize 1x1って何。

    else:
      action = torch.LongTensor([[random.randrange(self.num_actions)]])
    
    return action

  def make_minibatch(self, episode):
    if episode < 30:
      transitions = self.memory.sample(BATCH_SIZE)
    else:
      indexes = self.td_error_memory.get_prioritized_indexes(BATCH_SIZE)
      transitions = [self.memory.memory[n] for n in indexes]
    
    batch = Transition(*zip(*transitions))
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

    return batch, state_batch, action_batch, reward_batch, non_final_next_states
  
  def get_expected_state_action_values(self):
    self.main_q_network.eval()
    self.target_q_network.eval()
    # self.model(state_batch) でbatch_size*2のテンソル(actionのQ値)が出る。
    # action_batchはbatch_size*(0, 1)のようになっている。
    # その後実行したactionの方のQ値を取り出すために、gatherを使っている。
    self.state_action_values = self.main_q_network(self.state_batch).gather(1, self.action_batch)

    # 次の状態(next_state)があるかどうかに注意しながら取り出し、max{Q(s_t+1, a)}を求める
    # ここではインデックスマスクを作る。...?
    # インデックスにByteTensor入れると、ByteTensorでTrueになってる要素を取り出して、そのそれぞれに対して代入を行えるっぽい。
    non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, self.batch.next_state)))

    next_state_values = torch.zeros(BATCH_SIZE)

    a_m = torch.zeros(BATCH_SIZE).type(torch.LongTensor)

    a_m[non_final_mask] = self.main_q_network(self.non_final_next_states).detach().max(1)[1]

    a_m_non_final_next_states = a_m[non_final_mask].view(-1, 1)

    # 次の状態をモデルに入力して、次の行動のQ値が大きいものを取り出す。
    # それをnext_state_valuesの対応するインデックスに代入していく。
    # 後々微分されないために（固定するために）detach()をする。くわしくはわからん。
    # 現在の状態についてのQ値は更新対象であるから、detachしないっぽい。
    next_state_values[non_final_mask] = self.target_q_network(self.non_final_next_states).gather(1, a_m_non_final_next_states).detach().squeeze()

    # 教師となるQ値を求める。説明で書いた式の右辺。
    expected_state_action_values = self.reward_batch + GAMMA * next_state_values

    return expected_state_action_values

  def update_main_q_network(self):
    self.main_q_network.train()

    # smooth_l1_lossはHuber関数
    # expected_state_action_valuesはsize: バッチサイズとなっているので、これをunsqueezeで(バッチサイズ*1)にする。...?
    loss = F.smooth_l1_loss(self.state_action_values, self.expected_state_action_values.unsqueeze (1))
    
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

  def update_target_q_network(self):
    self.target_q_network.load_state_dict(self.main_q_network.state_dict())
  
  def update_td_error_memory(self):
    ''' TD誤差の更新 '''
    self.main_q_network.eval()
    self.target_q_network.eval()

    transitions = self.memory.memory
    batch = Transition(*zip(*transitions))

    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

    state_action_values = self.main_q_network(state_batch).gather(1, action_batch)

    non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))

    next_state_values = torch.zeros(len(self.memory))
    a_m = torch.zeros(len(self.memory)).type(torch.LongTensor)

    a_m[non_final_mask] = self.main_q_network(non_final_next_states).detach().max(1)[1]

    # 次の状態があるものだけに絞り、32を32x1にする。
    a_m_non_final_next_states = a_m[non_final_mask].view(-1, 1)

    # それをnext_state_valuesの対応するインデックスに代入していく。
    # 次の状態をモデルに入力して、次の行動のQ値が大きいものを取り出す。
    # 後々微分されないために（固定するために）detach()をする。くわしくはわからん。
    # 現在の状態についてのQ値は更新対象であるから、detachしないっぽい。
    next_state_values[non_final_mask] = self.target_q_network(non_final_next_states).gather(1, a_m_non_final_next_states).detach().squeeze()
    td_errors = (reward_batch + GAMMA * next_state_values) - state_action_values.squeeze()

    self.td_error_memory.memory = td_errors.detach().numpy().tolist()


In [ ]:
class Agent:
  def __init__(self, num_states, num_actions):
    self.brain = Brain(num_states, num_actions)
  
  def update_q_function(self, episode):
    self.brain.replay(episode)
  
  def get_action(self, state, episode):
    action = self.brain.decide_action(state, episode)
    return action

  def memorize(self, state, action, state_next, reward):
    self.brain.memory.push(state, action, state_next, reward)

  def update_target_q_function(self):
    self.brain.update_target_q_network()
  
  def memorize_td_error(self, td_error):
    self.brain.td_error_memory.push(td_error)
  
  def update_td_error_memory(self):
    self.brain.update_td_error_memory()


In [ ]:
class Environment:
  def __init__(self):
    self.env = wrap_env(gym.make(ENV))
    self.num_states = self.env.observation_space.shape[0]
    self.num_actions = self.env.action_space.n

    self.agent = Agent(self.num_states, self.num_actions)
  
  def run(self):
    # 10回ごとに立ち続けたepisodeの平均を取る
    episode_10_list = np.zeros(10)
    # 連続成功記録
    complete_episodes = 0
    episode_final = False

    for episode in range(NUM_EPISODES):
      observation = self.env.reset()

      state = observation
      state = torch.from_numpy(state).type(torch.FloatTensor)
      state = torch.unsqueeze(state, 0)

      for step in range(MAX_STEPS):
        
        action = self.agent.get_action(state, episode)

        observation_next, _, done, _ = self.env.step(action.item())

        if done:
          state_next = None
          episode_10_list = np.hstack((episode_10_list[1:], step + 1))

          if step < 195:
            reward = torch.FloatTensor([-1.0])
            complete_episodes = 0
          else:
            reward = torch.FloatTensor([1.0])
            complete_episodes += 1
        else:
          reward = torch.FloatTensor([0.0])
          state_next = observation_next
          state_next = torch.from_numpy(state_next).type(torch.FloatTensor)
          state_next = torch.unsqueeze(state_next, 0)
        
        self.agent.memorize(state, action, state_next, reward)

        self.agent.memorize_td_error(0)

        self.agent.update_q_function(episode)

        state = state_next

        if done:
          print(f'{episode} Episode: Finished after {step + 1} time steps: 10思考の平均step数 = {episode_10_list.mean()}')

          if episode % 2 == 0:
            self.agent.update_target_q_function()
          break
        
      if episode_final is True:
        show_video()
        break
      
      if complete_episodes >= 10:
        print('10回連続成功')
        episode_final = True


In [ ]:
from collections import namedtuple
Tr = namedtuple('tr', ('name_a', 'value_b'))
Tr_object = Tr('名前Aです', 100)
print(Tr_object)
print(Tr_object.value_b)
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


ENV = 'CartPole-v0'
GAMMA = 0.99
MAX_STEPS = 200
NUM_EPISODES = 500

tr(name_a='名前Aです', value_b=100)
100


In [ ]:
cartpole_env = Environment()
cartpole_env.run()

Net(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Episode: Finished after 18 time steps: 10思考の平均step数 = 1.8
1 Episode: Finished after 11 time steps: 10思考の平均step数 = 2.9
2 Episode: Finished after 14 time steps: 10思考の平均step数 = 4.3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


3 Episode: Finished after 13 time steps: 10思考の平均step数 = 5.6
4 Episode: Finished after 20 time steps: 10思考の平均step数 = 7.6
5 Episode: Finished after 17 time steps: 10思考の平均step数 = 9.3
6 Episode: Finished after 22 time steps: 10思考の平均step数 = 11.5
7 Episode: Finished after 18 time steps: 10思考の平均step数 = 13.3
8 Episode: Finished after 15 time steps: 10思考の平均step数 = 14.8
9 Episode: Finished after 15 time steps: 10思考の平均step数 = 16.3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


10 Episode: Finished after 22 time steps: 10思考の平均step数 = 16.7
11 Episode: Finished after 39 time steps: 10思考の平均step数 = 19.5
12 Episode: Finished after 24 time steps: 10思考の平均step数 = 20.5
13 Episode: Finished after 21 time steps: 10思考の平均step数 = 21.3
14 Episode: Finished after 88 time steps: 10思考の平均step数 = 28.1
15 Episode: Finished after 9 time steps: 10思考の平均step数 = 27.3
16 Episode: Finished after 13 time steps: 10思考の平均step数 = 26.4
17 Episode: Finished after 25 time steps: 10思考の平均step数 = 27.1
18 Episode: Finished after 22 time steps: 10思考の平均step数 = 27.8
19 Episode: Finished after 9 time steps: 10思考の平均step数 = 27.2


...

350 Episode: Finished after 200 time steps: 10思考の平均step数 = 190.4
351 Episode: Finished after 199 time steps: 10思考の平均step数 = 191.8
352 Episode: Finished after 200 time steps: 10思考の平均step数 = 194.0
353 Episode: Finished after 193 time steps: 10思考の平均step数 = 196.0
354 Episode: Finished after 196 time steps: 10思考の平均step数 = 198.0
355 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.0
356 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.0
357 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.0
358 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.0
359 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.8


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


360 Episode: Finished after 180 time steps: 10思考の平均step数 = 196.8
361 Episode: Finished after 200 time steps: 10思考の平均step数 = 196.9
362 Episode: Finished after 200 time steps: 10思考の平均step数 = 196.9
363 Episode: Finished after 196 time steps: 10思考の平均step数 = 197.2
364 Episode: Finished after 187 time steps: 10思考の平均step数 = 196.3
365 Episode: Finished after 200 time steps: 10思考の平均step数 = 196.3
366 Episode: Finished after 200 time steps: 10思考の平均step数 = 196.3
367 Episode: Finished after 185 time steps: 10思考の平均step数 = 194.8
368 Episode: Finished after 200 time steps: 10思考の平均step数 = 194.8
369 Episode: Finished after 200 time steps: 10思考の平均step数 = 194.8


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


370 Episode: Finished after 200 time steps: 10思考の平均step数 = 196.8
371 Episode: Finished after 200 time steps: 10思考の平均step数 = 196.8
372 Episode: Finished after 200 time steps: 10思考の平均step数 = 196.8
373 Episode: Finished after 200 time steps: 10思考の平均step数 = 197.2
374 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.5
375 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.5
376 Episode: Finished after 197 time steps: 10思考の平均step数 = 198.2
377 Episode: Finished after 200 time steps: 10思考の平均step数 = 199.7
10回連続成功
378 Episode: Finished after 181 time steps: 10思考の平均step数 = 197.8
